In [ ]:
#Setting all the input variables

storm_name = "dorian"
storm_year = 2019
storm_lowerLat= 40

latExtension = 2
lonExtension = 2
dateExtension = 2

focus_variable = "wind_spd_avg"
var_units = "(m s-1)"

#interest_variables = ["wind_spd", "wave_ht", "pressure"]
# Categories: Wind, Surface Waves, Pressure

In [ ]:
# Getting storm data and matching ERDDAP datasets around the storm (time and space)

import tropycal.tracks as tracks
import pandas as pd
from erddapy import ERDDAP
from datetime import timedelta

basin = tracks.TrackDataset(basin='north_atlantic', source='ibtracs')
storm = basin.get_storm((storm_name,storm_year))

# Coordinate selector throws an error when using with Fiona but still generates the graph
storm = storm.sel(lat=[storm_lowerLat,None])

storm_df = storm.to_dataframe()

min_lat = storm_df['lat'].min()
min_lon = storm_df['lon'].min()
max_lat = storm_df['lat'].max()
max_lon = storm_df['lon'].max()
start_date = storm_df.min()['date']
end_date = storm_df.max()['date']

# Get datasets matching time and location criteria, plus a buffer zone

e = ERDDAP(
    server="https://cioosatlantic.ca/erddap", 
    protocol="tabledap",
    response="csv"
)

e.dataset_id = "allDatasets"

e.variables = [
    "datasetID",
    "minLongitude",
    "maxLongitude",
    "minLatitude",
    "maxLatitude",
    "minTime",
    "maxTime"
]

e.constraints = {
    "minTime<=": end_date + timedelta(days = dateExtension),
    "maxTime>=": start_date- timedelta(days = dateExtension),
    "minLatitude>=": min_lat - latExtension,
    "maxLatitude<=": max_lat + latExtension,
    "minLongitude>=": min_lon - lonExtension,
    "maxLongitude<=": max_lon + lonExtension,
}

datasets_df = e.to_pandas(
    parse_dates=True,
).dropna()
datasets = set(datasets_df["datasetID"])

In [ ]:
# Narrow down datasets that contain the focused variable
# Some datasets may not match at certain points if their coverage is inconsistent

matching_datasets = []
for dataset in datasets:
    info_url = e.get_info_url(dataset_id = dataset, response = "csv")
    dataset_info = pd.read_csv(info_url)

    variable_names = set(dataset_info["Variable Name"].unique())
    if focus_variable in variable_names:
         matching_datasets.append(dataset)

print(matching_datasets)

In [ ]:
# Find most recent values for focus var given a time point

selected_time=  start_date + timedelta(days = 1)
matching_points = pd.DataFrame()

e = ERDDAP(
    server="https://cioosatlantic.ca/erddap", 
    protocol="tabledap",
    response="csv",
)

for dataset in matching_datasets:

    e.dataset_id = dataset
    print(dataset)
    
    e.constraints = {
        "time<=": selected_time,
        "time>=": selected_time - timedelta(hours = 6)
    }
        
    e.variables = ["time", "longitude", "latitude", focus_variable]

    try:
        buoy_data= e.to_pandas(
            parse_dates=True,
        ).dropna()
        recent_row = buoy_data.tail(1)
        print(recent_row["time (UTC)"].iloc[0])
        print(recent_row[focus_variable + " " + var_units].iloc[0])
        recent_row.insert(0, "dataset", dataset)
        matching_points = pd.concat([matching_points, recent_row.loc[:]]).reset_index(drop=True)
    except:
        print("couldn't find info for the given dataset for the given time")
    print("-")


In [ ]:
matching_points.set_index('dataset')
print(matching_points)

geo_plot = matching_points.plot(kind='scatter', x='longitude (degrees_east)', y='latitude (degrees_north)', 
                                c='wind_spd_avg (m s-1)', colormap = 'viridis_r')
for idx, row in matching_points.iterrows():
    geo_plot.annotate(idx,[row['longitude (degrees_east)'], row['latitude (degrees_north)']])

In [ ]:
# Create plots  for matching datasets
e = ERDDAP(
    server="https://cioosatlantic.ca/erddap", 
    protocol="tabledap",
    response="csv",
)

for dataset in matching_datasets:

    # Still keeps generating graph after dataset fails for some reason?

    e.dataset_id = dataset

    e.constraints = {
        "time>=": start_date - timedelta(days = 2),
        "time<=": end_date + timedelta(days = 2)
    }

    e.variables = ["time", "longitude", "latitude", focus_variable]

    try:
        buoy_data= e.to_pandas(
            parse_dates=True,
        ).dropna()
        buoy_data.plot(x='time (UTC)', y=focus_variable + ' ' + var_units, title=dataset)
    except:
        print("Data does not exist for %s", dataset) 
